# PLAN

0. [x] Get the data.
0. [x] Prepare the data for exploration.
0. [x] Split the data into train and test sets.
0. [ ] Explore the data.
0. [ ] Do cross-validation with the data.
0. [ ] Code and evaluate several models.
0. [ ] Evaluate out of sample data (test set).

# ENVIRONMENT

In [1]:
import os
import acquire
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


# import xgboost as xgb

from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

# conda install -c glemaitre imbalanced-learn
from imblearn.ensemble import BalancedBaggingClassifier

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# ACQUISITION

In [2]:
df1 = acquire.read_data('data01.csv')
df2 = acquire.read_data('data02.csv')
df3 = acquire.read_data('data03.csv')
df4 = acquire.read_data('data04.csv')
df5 = acquire.read_data('data05.csv')
df7 = acquire.read_data('data07.csv')

In [3]:
# df1 = acquire.join_data('data01.csv', 'data02.csv')
# df2 = acquire.join_data('data03.csv', 'data04.csv')
# df3 = acquire.join_data('data05.csv', 'data07.csv')

In [4]:
dfs = [df1, df2, df3, df4, df5, df7]
# dfa = reduce(lambda left,right: pd.merge(left,right,on='CASEID'), dfs)

In [5]:
# Specify columns to take from dataframes 4, 5, 7:
four_cols = ['CASEID', 
             'G1NUMBER', 
             'H1JEALUS', 
             'H2LIMIT',
           'H3KNOWNG',
           'J1HIT',
           'J2THROWN',
           'J3PUSH',
           'J4SLAP',
           'J5KICK',
           'J6OBJECT',
           'J7BEAT',
           'J8CHOKE',
           'J9KNIFE',
           'J10GUN',
           'J11SEX',
           'POWER',
           'HARASS']
five_cols = ['CASEID', 
             'B1AGE', 
            'AGEDISP',
            'STDETAI']
sev_cols = ['CASEID', 
            'SAMESEXR',
           'N11DRUGS',
           'N12ALCHL',
           'N13SUHIM',
           'N16CHILD',
           'N17ARRST',
           'N1FRQNCY',
           'N2SVRITY',
           'N3WEAPON',
           'N4CHOKE',
           'N5SEX',
           'N6CONTRL',
           'N8JEALUS',
           'N10CPBLE']

In [6]:
# drop dataframes down to desired features
df4 = df4[four_cols]
df5 = df5[five_cols]
df7 = df7[sev_cols]

In [7]:
#take a look at new dataframes
df4.head()

,CASEID,G1NUMBER,H1JEALUS,H2LIMIT,H3KNOWNG,J1HIT,J2THROWN,J3PUSH,J4SLAP,J5KICK,J6OBJECT,J7BEAT,J8CHOKE,J9KNIFE,J10GUN,J11SEX,POWER,HARASS
0,108395,9999,1,2,1,1,1,1,2,2,2,2,2,2,2,2,2,3
1,108397,9999,1,2,1,1,2,1,1,2,2,2,1,1,2,2,2,5
2,108399,9999,1,2,1,2,1,1,2,2,2,2,1,2,2,2,3,2
3,108443,9999,1,2,1,1,1,1,1,2,1,2,1,2,2,2,3,5
4,108444,9999,1,2,1,1,2,1,1,2,2,2,2,2,2,2,2,3


In [8]:
# take a look at no2
df5.head()

,CASEID,B1AGE,AGEDISP,STDETAI
0,108395,18,4,1
1,108397,30,4,2
2,108399,27,1,1
3,108443,19,1,2
4,108444,18,1,5


In [9]:
# take a look at no3
df7.head()

,CASEID,SAMESEXR,N11DRUGS,N12ALCHL,N13SUHIM,N16CHILD,N17ARRST,N1FRQNCY,N2SVRITY,N3WEAPON,N4CHOKE,N5SEX,N6CONTRL,N8JEALUS,N10CPBLE
0,108395,9999,1,1,2,2,2,2,2,2,2,2,2,2,2
1,108397,9999,2,1,2,2,1,2,1,1,1,1,2,1,1
2,108399,9999,2,2,2,2,2,2,2,2,1,2,2,2,2
3,108443,9999,2,1,2,2,1,2,2,1,1,2,1,2,2
4,108444,9999,2,2,2,2,1,1,1,2,2,2,2,2,2


In [10]:
# first merge of df4 to df5
dfn = df4.merge(right=df5, on='CASEID')
# second merge of df4+5 to df7
dfn = dfn.merge(right=df7, on='CASEID')

In [11]:
# establish shape
dfn.shape

(705, 35)

In [12]:
# establish unique caseid no's
dfn.CASEID.nunique()

705

In [13]:
# # applying filter function  
# dfa.filter(['ABUSED',
#            'SCRSTATR',
#            'LENGTHC1',
#            'C1SITUAT',
#            'PABUSE',
#            'D3RCHILT',
#            'E13PRGNT',
#            'N7PREGNT',
#            'TOTSUPRT',
#            'G1NUMBER',
#            'H1JEALUS',
#            'H2LIMIT',
#            'H3KNOWNG',
#            'J1HIT',
#            'J2THROWN',
#            'J3PUSH',
#            'J4SLAP',
#            'J5KICK',
#            'J6OBJECT',
#            'J7BEAT',
#            'J8CHOKE',
#            'J9KNIFE',
#            'J10GUN',
#            'J11SEX',
#            'POWER',
#            'HARASS',
#            'SAMESEXR',
#            'N11DRUGS',
#            'N12ALCHL',
#            'N13SUHIM',
#            'N16CHILD',
#            'N17ARRST',
#            'N1FRQNCY',
#            'N2SCRITY',
#            'N3WEAPON',
#            'N4CHOKE',
#            'N5SEX',
#            'N6CONTRL',
#            'N8JEALUS',
#            'N10CPBLE',
#           ])

In [14]:
dfn.to_csv("data/2nd_half.csv", index=False)